In [1]:
# !jupyter nbextension enable --py gmaps
# !jupyter nbextension enable --py widgetsnbextension

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from api_keys import g_key

In [3]:
df = pd.read_csv("resources/city_weather.csv")
df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mahenge,-8.6833,36.7167,62.85,97,44,1.54,TZ,1669854193
1,1,shaunavon,49.6501,-108.4181,6.58,85,100,19.39,CA,1669854193
2,2,saint-gaudens,43.0833,0.6667,35.02,75,20,3.74,FR,1669854194
3,3,osmena,10.5208,119.9258,82.38,79,88,20.38,PH,1669854194
4,4,ordzhonikidze,52.4411,61.7408,-0.87,84,15,10.71,KZ,1669854195


In [4]:
gmaps.configure(api_key=g_key)


In [5]:
# Heatmap of humidity
locations = df[["Lat", "Lng"]]
humidity = df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

In [6]:
# filter for "ideal" weather (70F-80F, windspeed<10mph, 0% cloudiness)
ideal_df = df.loc[(df["Max Temp"] < 80) & (df["Max Temp"] > 70) \
                                    & (df["Wind Speed"] < 10) \
                                    & (df["Cloudiness"] < 20)].dropna()
ideal_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
140,140,port augusta,-32.5000,137.7667,76.12,32,17,6.96,AU,1669854232
150,150,pedernales,18.0384,-71.7440,78.53,77,16,3.96,DO,1669854235
192,192,mitsamiouli,-11.3847,43.2844,77.74,76,1,4.61,KM,1669854246
207,207,bahia honda,22.9036,-83.1592,77.05,88,14,9.46,CU,1669854250


In [7]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel_df = ideal_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
140,port augusta,AU,-32.5000,137.7667,
150,pedernales,DO,18.0384,-71.7440,
192,mitsamiouli,KM,-11.3847,43.2844,
207,bahia honda,CU,22.9036,-83.1592,


In [8]:
 # Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    address = requests.get(base_url, params=params)
    
    # convert to json
    address = address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = address["results"][0]["name"]
    except (KeyError, IndexError):
        print("No results... Skip to next.")

hotel_df

,City,Country,Lat,Lng,Hotel Name
140,port augusta,AU,-32.5000,137.7667,Majestic Oasis Apartments
150,pedernales,DO,18.0384,-71.7440,Hotel Costa Marina
192,mitsamiouli,KM,-11.3847,43.2844,Foyer ADM
207,bahia honda,CU,22.9036,-83.1592,Casa Gualberto - Deysi


In [9]:
# create template for mapping hotel markers
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
locations

,Lat,Lng
140,-32.5000,137.7667
150,18.0384,-71.7440
192,-11.3847,43.2844
207,22.9036,-83.1592


In [11]:
fig2 = fig

In [12]:
# Add marker layer for heatmap
hotel_markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig2.add_layer(hotel_markers)
fig2

Figure(layout=FigureLayout(height='420px'))